In [4]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from datetime import datetime

from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    plot_edge_color_value,
    reconstruct_image_from_general,
    plot_full,
    find_nearest_edge,
)
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    # get_width_info,
    get_width_info_new,
)
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
)
import pandas as pd
from amftrack.pipeline.functions.spore_processing.spore_id import make_spore_data
from amftrack.pipeline.functions.image_processing.hyphae_id_surf import (
    resolve_anastomosis_crossing_by_root,
)
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Node,
    Edge,
    Hyphae,
)
from datetime import datetime
from matplotlib import cm
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from IPython.display import clear_output
from amftrack.pipeline.functions.post_processing.exp_plot import *
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    run_launcher,
    run_parallel_all_time,
)

plates = [
    "441_20230807","449_20230807","310_20230830"
]
directory_targ = "/projects/0/einf914/transport/"
plates = [
 #    "393_20231129",
	# "366_20230921",
	# "3_20220426",
	# "1045_20220504",
	# "559_20230109",
	# "219_20230307",
	"252_20230316",
]
directory_targ = "/projects/0/einf914/john/"
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_width.p"] == True]
limits = {
        "393_20231129": (0, 99),
	"366_20230921": (0, 68),
 
	"3_20220426": (77, 141),
 
	"1045_20220504": (4, 71),
	"559_20230109" : (0, 101),
 
	"219_20230307": (0, 59),
 
	"252_20230316": (0, 84),
}

import pickle
import scipy.io as sio

path_root = f"/scratch-shared/amftrack/graph_stacks"

for plate_id in plates:
    # update_plate_info(directory_targ, local=True,strong_constraint=False)
    # all_folders = get_current_folders(directory_targ, local=True)
    folders = all_folders.loc[all_folders["unique_id"] == plate_id]
    folders = folders.loc[folders["/Analysis/nx_graph_pruned_width.p"] == True]
    folders = folders.sort_values(by="folder")
    
    folders = folders.sort_values(by="datetime")
    limits_id = limits[plate_id]
    folders = folders.iloc[limits_id[0]:limits_id[1]]
    exp = Experiment(directory_targ)
    # i = indexes[plate_id_video]
    # i = np.where(folders['folder'] == indexes[plate_id_video])[0][0]
    # selection = folders[folders['folder'].isin(indexes.values())]
    for i in range(1,len(folders)):
        selection =folders.iloc[i : i + 1]
        folder_list = list(selection["folder"])
        directory_name = folder_list[0]
        path_snap = directory_targ + directory_name
        transform = sio.loadmat(path_snap + "/Analysis/transform_new.mat")
        exp.load(selection, suffix="_realigned")
        for t in range(exp.ts):
            exp.load_tile_information(t)
            exp.save_location = ""
            
            load_study_zone(exp)
        os.makedirs(os.path.join(path_root,plate_id,),exist_ok = True)
        graph = exp.nx_graph[0]
        graph_to_save = graph.copy()
        for u, v, data in graph_to_save.edges(data=True):
            data["pixels"] = data.pop("pixel_list")
        node_not_in_ROI = []
        for node in graph_to_save:
            if not is_in_ROI_node(Node(node, exp), 0):
                node_not_in_ROI.append(node)
        graph_to_save.remove_nodes_from(node_not_in_ROI)
        nx.set_node_attributes(graph_to_save, exp.positions[0], 'position')
        path_tot = os.path.join(path_root,plate_id,f"graph{i:03d}.pickle")
        pickle.dump((graph_to_save,transform,selection.iloc[0]), open(path_tot, 'wb'))
        break

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


analysed:   0%|          | 0/1676 [00:00<?, ?it/s]

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/pipeline/functions/image_processing/experiment_class_surf.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.folders["datetime"] = pd.to_datetime(


2023-03-16 20:48:00


In [2]:
folders

,total_path,Plate,PrincePos,root,strain,medium,fungalC,split,Pbait,CrossDate,...,/Analysis/skeleton_pruned_realigned.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,/Analysis/nx_graph_pruned_labeled2.p,date,folder,pH,unique_id,datetime
213,/projects/0/einf914/john/20230307_1522_Plate11,219,11,'Carrot','C3','001P100N100C',NaN,'Y','-',20230307,...,True,True,True,True,True,"07.03.2023, 15:22:",20230307_1522_Plate11,NaN,219_20230307,2023-03-07 15:22:00
1472,/projects/0/einf914/john/20230307_1723_Plate11,219,11,'Carrot','C3','001P100N100C',NaN,'Y','-',20230307,...,True,True,True,True,True,"07.03.2023, 17:23:",20230307_1723_Plate11,NaN,219_20230307,2023-03-07 17:23:00
1052,/projects/0/einf914/john/20230307_1920_Plate11,219,11,'Carrot','C3','001P100N100C',NaN,'Y','-',20230307,...,True,True,True,True,True,"07.03.2023, 19:20:",20230307_1920_Plate11,NaN,219_20230307,2023-03-07 19:20:00
1093,/projects/0/einf914/john/20230307_2119_Plate11,219,11,'Carrot','C3','001P100N100C',NaN,'Y','-',20230307,...,True,True,True,True,True,"07.03.2023, 21:19:",20230307_2119_Plate11,NaN,219_20230307,2023-03-07 21:19:00
52,/projects/0/einf914/john/20230307_2319_Plate11,219,11,'Carrot','C3','001P100N100C',NaN,'Y','-',20230307,...,True,True,True,True,True,"07.03.2023, 23:19:",20230307_2319_Plate11,NaN,219_20230307,2023-03-07 23:19:00
952,/projects/0/einf914/john/20230308_0119_Plate11,219,11,'Carrot','C3','001P100N100C',NaN,'Y','-',20230307,...,True,True,True,True,True,"08.03.2023, 01:19:",20230308_0119_Plate11,NaN,219_20230307,2023-03-08 01:19:00
1081,/projects/0/einf914/john/20230308_0319_Plate11,219,11,'Carrot','C3','001P100N100C',NaN,'Y','-',20230307,...,True,True,True,True,True,"08.03.2023, 03:19:",20230308_0319_Plate11,NaN,219_20230307,2023-03-08 03:19:00
1520,/projects/0/einf914/john/20230308_0520_Plate11,219,11,'Carrot','C3','001P100N100C',NaN,'Y','-',20230307,...,True,True,True,True,True,"08.03.2023, 05:20:",20230308_0520_Plate11,NaN,219_20230307,2023-03-08 05:20:00
452,/projects/0/einf914/john/20230308_0720_Plate11,219,11,'Carrot','C3','001P100N100C',NaN,'Y','-',20230307,...,True,True,True,True,True,"08.03.2023, 07:20:",20230308_0720_Plate11,NaN,219_20230307,2023-03-08 07:20:00
1273,/projects/0/einf914/john/20230308_0920_Plate11,219,11,'Carrot','C3','001P100N100C',NaN,'Y','-',20230307,...,True,True,True,True,True,"08.03.2023, 09:20:",20230308_0920_Plate11,NaN,219_20230307,2023-03-08 09:20:00


In [2]:
folders

,total_path,Plate,PrincePos,root,strain,medium,fungalC,split,Pbait,CrossDate,...,/Analysis/skeleton_pruned_realigned.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,/Analysis/nx_graph_pruned_labeled2.p,date,folder,pH,unique_id,datetime
1426,/projects/0/einf914/john/20230109_1448_Plate18,559,18,'Carrot','C2','001P100N100C',NaN,'Y','-',20230109,...,True,True,True,True,True,"09.01.2023, 14:48:",20230109_1448_Plate18,NaN,559_20230109,2023-01-09 14:48:00
463,/projects/0/einf914/john/20230109_1638_Plate18,559,18,'Carrot','C2','001P100N100C',NaN,'Y','-',20230109,...,True,True,True,True,True,"09.01.2023, 16:38:",20230109_1638_Plate18,NaN,559_20230109,2023-01-09 16:38:00
330,/projects/0/einf914/john/20230109_1837_Plate18,559,18,'Carrot','C2','001P100N100C',NaN,'Y','-',20230109,...,True,True,True,True,True,"09.01.2023, 18:37:",20230109_1837_Plate18,NaN,559_20230109,2023-01-09 18:37:00
1156,/projects/0/einf914/john/20230109_2037_Plate18,559,18,'Carrot','C2','001P100N100C',NaN,'Y','-',20230109,...,True,True,True,True,True,"09.01.2023, 20:37:",20230109_2037_Plate18,NaN,559_20230109,2023-01-09 20:37:00
1436,/projects/0/einf914/john/20230109_2237_Plate18,559,18,'Carrot','C2','001P100N100C',NaN,'Y','-',20230109,...,True,True,True,True,True,"09.01.2023, 22:37:",20230109_2237_Plate18,NaN,559_20230109,2023-01-09 22:37:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,/projects/0/einf914/john/20230117_1433_Plate18,559,18,'Carrot','C2','001P100N100C',NaN,'Y','-',20230109,...,True,True,True,True,True,"17.01.2023, 14:33:",20230117_1433_Plate18,NaN,559_20230109,2023-01-17 14:33:00
801,/projects/0/einf914/john/20230117_1634_Plate18,559,18,'Carrot','C2','001P100N100C',NaN,'Y','-',20230109,...,True,True,True,True,True,"17.01.2023, 16:34:",20230117_1634_Plate18,NaN,559_20230109,2023-01-17 16:34:00
332,/projects/0/einf914/john/20230117_1834_Plate18,559,18,'Carrot','C2','001P100N100C',NaN,'Y','-',20230109,...,True,True,True,True,True,"17.01.2023, 18:34:",20230117_1834_Plate18,NaN,559_20230109,2023-01-17 18:34:00
613,/projects/0/einf914/john/20230117_2034_Plate18,559,18,'Carrot','C2','001P100N100C',NaN,'Y','-',20230109,...,True,True,True,True,True,"17.01.2023, 20:34:",20230117_2034_Plate18,NaN,559_20230109,2023-01-17 20:34:00
